<img src='sharif_logo.png' alt="SUT logo" width=150 height=150 align=left class="saturate" >

<br>
<font face="Times New Roman">
<div dir=ltr align=center>
<font color=0F5298 size=7>
 Deep Learning <br>
<font color=2565AE size=5>
Computer Engineering Department - Spring 2025  <br>
<font color=3C99D size=5>
          Homework 2:  <br>
<font color=696880 size=4>
           
    

# Assignment Overview

In this assignment, you will explore inference scaling techniques in large language models (LLMs) and evaluate their performance using the Math Benchmark. Throughout the notebook, you will learn about several inference methods, including:

- **Chain-of-Thought (CoT):** A method where the model generates intermediate reasoning steps before providing the final answer.
- **Best-of-n Sampling:** An approach that generates multiple candidate responses and selects the best one based on a scoring function.
- **Beam Search:** A technique that expands several possible sequences simultaneously, choosing the most promising ones based on probability.
- **Self-Refinement:** An iterative process where the model revises its output to improve accuracy and coherence.

The **Math Benchmark** is a suite of challenging mathematical problems designed to test the reasoning and problem-solving capabilities of LLMs. The benchmark includes a variety of questions ranging from basic arithmetic and algebra to more advanced topics such as geometry and calculus. For example, you might be asked to solve an equation like `2x + 5 = 15` or compute the derivative of a function, tasks that assess the model's ability to handle both straightforward and complex mathematical queries.

By the end of this assignment, you will have:
- Gained a deeper understanding of inference time scaling methods in LLMs.
- Compared the effectiveness of different inference techniques using a rigorous math evaluation framework.

Let's dive into the notebook and begin exploring how these methods perform on a challenging set of math problems!


## vLLM: Accelerated Inference Engine for LLMs

vLLM is an open-source project designed to optimize the loading and inference of large language models. By leveraging advanced memory management techniques and dynamic batching, vLLM significantly speeds up the inference process, making it easier to deploy and experiment with LLMs even on hardware with limited resources
So we use vLLM to get results faster.

# installing Dependencies

In [ ]:
!pip install vllm

In [ ]:
!pip install transformers accelerate datasets

In [ ]:
!pip install --upgrade numpy
import os
os.kill(os.getpid(), 9)


This command launches a vLLM inference server with:
- Model: `DeepSeek-R1-Distill-Qwen-1.5B`
- Port: `8000` (default API endpoint)
- Precision: `half` (FP16) for memory efficiency
- Max context length: `3192` tokens

**Note:**  
🔹 Ensure you're using a GPU runtime (T4 or better) in Colab  
🔹 Only run the next cell if this one executes successfully 


In [ ]:
!vllm serve "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"   --port 8000   --dtype=half   --max-model-len 3192

* this cell lunches model in background using vllm

In [ ]:
!nohup vllm serve "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B" --port 8000 --dtype=half --max-model-len 5192 &

## LLM Query Function

* This Python function sends prompts to a locally-hosted LLM API and returns the generated response
* you can change max_tokens and temperature as you want



In [4]:
import requests
def get_llm_response(prompt):
    url = "http://localhost:8000/v1/chat/completions"

    payload = {
        "model": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }

        ],
    "max_tokens": 500,
    "temperature": 0.6
    }
    response = requests.post(url, json=payload)
    return response.json()['choices'][0]['message']['content'].strip()

# Test response generation
- testing model with some Math benchmark quesions

In [ ]:

# Simple generation of sample math questions
print("Q1:", question1)
resp1 = get_llm_response(question1)
print(resp1, "
")

print("Q2:", question2)
resp2 = get_llm_response(question2)
print(resp2, "
")

print("Q3:", question3)
resp3 = get_llm_response(question3)
print(resp3)


# Math Benchmark Evaluation

This cell is dedicated to evaluating the performance of inference scaling methods on the Math Benchmark dataset. The process works as follows:

- **Dataset Loading:** It loads the MATH-500 dataset, which contains a set of challenging math problems along with their correct solutions.
- **Answer Extraction:** The `extract_answer` function is used to parse and extract the final answer from the generated responses. This function specifically looks for a LaTeX-style format (using `\boxed{...}`) to reliably pinpoint the answer.
- **Normalization and Comparison:** Before comparing, both the predicted answer and the ground truth are normalized using several functions. These functions handle different mathematical expressions, such as fractions, matrices, and algebraic expressions, ensuring that the comparison is fair and accurate regardless of formatting differences.
- **Evaluation Loop:** For each problem:
  - The ground truth answer is extracted from the provided solution.
  - A response is generated by the LLM using a designated function.
  - The predicted answer is then extracted and compared against the ground truth.
  - The results for each problem, including whether the predicted answer is correct, are saved for later analysis.
- **Results Analysis:** After processing all problems, the cell aggregates the results and prints a summary, including the total number of problems evaluated, the number of correct answers, and the overall accuracy.

This evaluation method ensures that the output of each inference technique (such as Chain-of-Thought, Best-of-n, Beam Search, and Self-Refinement) is consistently measured against the Math Benchmark, without altering the original answers or evaluation logic.

**Note:**  

🔹 you don't need to modify this cell. Only rewrite the evaluation function portion then

🔹 you need to run this cell before evaluating.


In [1]:
import json
import os
import re
from typing import Dict, Optional, Union
from datasets import load_dataset
from tqdm import tqdm
import torch

# Load the MATH-500 dataset
def load_math500_dataset():
    dataset = load_dataset("HuggingFaceH4/MATH-500")["test"]
    return dataset

# Extract the last boxed answer from text
def extract_answer(response: str) -> Optional[str]:
    if not response:
        return None
    start_idx = response.rfind('\\boxed{')
    if start_idx == -1:
        return None
    brace_count = 1
    pos = start_idx + 7  # length of '\boxed{'
    while pos < len(response) and brace_count > 0:
        if response[pos] == '{':
            brace_count += 1
        elif response[pos] == '}':
            brace_count -= 1
        pos += 1
    if brace_count == 0:
        answer = response[start_idx + 7:pos - 1]
        return answer.strip()
    return None

# Normalization and comparison functions (unchanged from original)
def normalize_number(num_str: str) -> str:
    try:
        cleaned = re.sub(r'[,\$\\]|\s*(?:cm|m|kg|ft|in|lb|oz|ml|L)$|\s*\\text{[^}]+}', '', num_str).strip()
        if cleaned.startswith('.'):
            cleaned = '0' + cleaned
        num = float(cleaned)
        if abs(num) < 1 and '.' in cleaned:
            decimal_places = len(cleaned.split('.')[1])
            format_str = f"{{:.{decimal_places}f}}"
            result = format_str.format(num)
        else:
            result = str(num)
        return result
    except:
        return num_str

def numerically_equal(str1: str, str2: str) -> bool:
    try:
        return abs(float(str1) - float(str2)) < 1e-10
    except:
        return False

def normalize_fraction(fraction_str: str) -> str:
    try:
        fraction_str = fraction_str.replace('\\dfrac', '\\frac')
        fraction_str = ''.join(fraction_str.split())
        fraction_str = re.sub(r'\s*\\text{[^}]+}', '', fraction_str)
        mixed_brace = re.match(r'^\\frac(\d+)\{(\d+)\}$', fraction_str)
        if mixed_brace:
            num, den = mixed_brace.groups()
            return f"\\frac{{{num}}}{{{den}}}"
        no_braces = re.match(r'^\\frac(\d+)(\d+)$', fraction_str)
        if no_braces:
            num, den = no_braces.groups()
            return f"\\frac{{{num}}}{{{den}}}"
        if '/' in fraction_str and not any(c in fraction_str for c in '\\{}'):
            num, den = fraction_str.split('/')
            return f"\\frac{{{num.strip()}}}{{{den.strip()}}}"
        standard = re.match(r'^\\frac\{([^{}]+)\}\{([^{}]+)\}$', fraction_str)
        if standard:
            num, den = standard.groups()
            return f"\\frac{{{num}}}{{{den}}}"
    except:
        return fraction_str

def normalize_matrix_entry(entry: str) -> str:
    entry = ''.join(entry.split())
    if '/' in entry and not any(c in entry for c in '\\{}'):
        if entry.startswith('-'):
            num, den = entry[1:].split('/')
            return f"-{num.strip()}/{den.strip()}"
        else:
            num, den = entry.split('/')
            return f"{num.strip()}/{den.strip()}"
    entry = entry.replace('\\dfrac', '\\frac')
    frac_match = re.match(r'^(-)?\\frac\{(\d+)\}\{(\d+)\}$', entry)
    if frac_match:
        sign, num, den = frac_match.groups()
        sign = sign if sign else ''
        return f"{sign}{num}/{den}"
    return entry

def normalize_matrix(matrix_str: str) -> str:
    try:
        matrix_str = ''.join(matrix_str.split())
        match = re.match(r'^\\begin\{pmatrix\}(.*?)\\end\{pmatrix\}$', matrix_str)
        if not match:
            return matrix_str
        content = match.group(1)
        rows = content.split('\\\\')
        normalized_rows = []
        for row in rows:
            if '&' in row:
                entries = [normalize_matrix_entry(entry) for entry in row.split('&')]
            else:
                entries = [normalize_matrix_entry(row)]
            normalized_rows.append('&'.join(entries))
        result = "\\begin{pmatrix}" + "\\\\".join(normalized_rows) + "\\end{pmatrix}"
        return result
    except:
        return matrix_str

def normalize_algebraic_expression(expr: str) -> str:
    try:
        expr = ''.join(expr.split())
        monomial_match = re.match(r'^(-?\d*\.?\d*)?([a-zA-Z])(?:\^(-?\d+))?$', expr)
        if monomial_match:
            coeff, var, exp = monomial_match.groups()
            coeff = coeff if coeff and coeff not in ['+', '-'] else ('1' if not coeff else '-1')
            exp = exp if exp else '1'
            if coeff == '1' and exp == '1':
                return var
            elif coeff == '1':
                return f"{var}^{exp}"
            elif coeff == '-1' and exp == '1':
                return f"-{var}"
            elif coeff == '-1':
                return f"-{var}^{exp}"
            elif exp == '1':
                return f"{coeff}{var}"
            else:
                return f"{coeff}{var}^{exp}"
        pi_term_match = re.match(r'^(-?\d*\.?\d*)\\?pi$', expr)
        if pi_term_match:
            coeff = pi_term_match.group(1)
            if not coeff or coeff == '-':
                coeff = '-1' if coeff == '-' else '1'
            return f"{coeff}\\pi"
        frac_pi_match = re.match(r'^\\frac{([^{}]+)}{([^{}]+)}\\?pi$', expr)
        if frac_pi_match:
            num, den = frac_pi_match.groups()
            return f"\\frac{{{num}}}{{{den}}}\\pi"
        frac_match = re.match(r'^\\frac{([^{}]+)}{([^{}]+)}$', expr)
        if frac_match:
            num, den = frac_match.groups()
            return f"\\frac{{{num}}}{{{den}}}"
    except:
        return expr.lower()

def normalize_interval_bound(bound: str) -> str:
    if '\\infty' in bound:
        sign = '-' if bound.startswith('-') else ''
        return f"{sign}\\infty"
    return normalize_answer(bound) or bound

def normalize_interval(interval_str: str) -> str:
    try:
        interval_str = ''.join(interval_str.split())
        match = re.match(r'^\\left?([\[\(])(.*?),(.*?)\\right?([\]\)])$', interval_str)
        if not match:
            match = re.match(r'^([\[\(])(.*?),(.*?)([\]\)])$', interval_str)
            if not match:
                return interval_str
        left_bracket, left_bound, right_bound, right_bracket = match.groups()
        norm_left = normalize_interval_bound(left_bound)
        norm_right = normalize_interval_bound(right_bound)
        return f"\\left{left_bracket}{norm_left},{norm_right}\\right{right_bracket}"
    except:
        return interval_str

def normalize_ordered_tuple(tuple_str: str) -> str:
    try:
        tuple_str = tuple_str.replace('\\dfrac', '\\frac')
        tuple_str = tuple_str.replace('\\left', '').replace('\\right', '')
        tuple_str = re.sub(r'\\?\s+', '', tuple_str)
        inner = tuple_str.strip('()')
        parts = inner.split(',')
        normalized_parts = [normalize_answer(part.strip()) for part in parts if normalize_answer(part.strip())]
        return f"({','.join(normalized_parts)})"
    except:
        return None

def normalize_answer(answer: str) -> str:
    if answer is None:
        return ""
    answer = re.sub(r'\\text{[^}]+(?:inches|feet|meters|cm|m|kg|ft|in|lb|oz|ml|L|per|second|minute|hour)[^}]*}', '', answer)
    answer = re.sub(r'(?<!\\)\s+', '', answer)
    ordered_pair_match = re.match(r'^(?:\\left)?\((.*?)(?:\\right)?\)$', answer)
    if ordered_pair_match:
        content = ordered_pair_match.group(1)
        parts = content.split(',')
        normalized_parts = [normalize_answer(part) for part in parts if normalize_answer(part)]
        return f"({','.join(normalized_parts)})"
    answer = ''.join(answer.split())
    if not answer:
        return None
    pm_match = re.match(r'^(.*?)(?:\\pm|-)(.*?)$', answer)
    if pm_match:
        left, right = pm_match.groups()
        norm_left = normalize_answer(left) if left else ""
        norm_right = normalize_answer(right) if right else ""
        if norm_left or norm_right:
            return f"{norm_left}\\pm{norm_right}"
    trig_match = re.match(r'^\\(?:sin|cos|tan|cot|sec|csc)\s*([a-zA-Z])$', answer)
    if trig_match:
        variable = trig_match.group(1)
        func_name = re.match(r'^\\(.*?)(?:\s|$)', answer).group(1)
        return f"\\{func_name}{variable}"
    text_match = re.match(r'^(?:\\text{)?([A-Za-z]+)(?:})?$', answer)
    if text_match:
        return text_match.group(1).lower()
    if (answer.startswith('\\left[') or answer.startswith('\\left(') or
        answer.startswith('[') or answer.startswith('(')) and \
       (answer.endswith('\\right]') or answer.endswith('\\right)') or
        answer.endswith(']') or answer.endswith(')')):
        return normalize_interval(answer)
    if answer.startswith('\\begin{pmatrix}') and answer.endswith('\\end{pmatrix}'):
        return normalize_matrix(answer)
    answer = answer.replace('\\dfrac', '\\frac')
    if '\\frac' in answer or '/' in answer:
        return normalize_fraction(answer)
    neg_sqrt_match = re.match(r'^-\\sqrt\{?(\d+)\}?$', answer)
    if neg_sqrt_match:
        num = neg_sqrt_match.group(1)
        return f"-\\sqrt{{{num}}}"
    sqrt_match = re.match(r'^(\d*)?\\sqrt\{?(\d+)\}?$', answer)
    if sqrt_match:
        coeff, num = sqrt_match.groups()
        coeff = coeff if coeff else '1'
        return f"\\sqrt{{{num}}}" if coeff == '1' else f"{coeff}\\sqrt{{{num}}}"
    sqrt_with_coeff_match = re.match(r'^(\d+)\\sqrt\{?(\d+)\}?$', answer)
    if sqrt_with_coeff_match:
        coeff, num = sqrt_with_coeff_match.groups()
        return f"{coeff}\\sqrt{{{num}}}"
    base_match = re.match(r'^(\d+)(?:_\{?(\d+)\}?|_(\d+))$', answer)
    if base_match:
        number, base1, base2 = base_match.groups()
        base = base1 if base1 else base2
        return f"{number}_{base}"
    percent_match = re.match(r'^(\d+(?:\.\d*)?)\s*\\?%$', answer)
    if percent_match:
        return normalize_number(percent_match.group(1))
    unit_match = re.match(r'^(\d+(?:\.\d*)?)\s*(?:(?:\\[,\s])|,)?\s*(?:\\\\)?(?:\\text{(\w+)}|\\?(?:cm|m|kg|ft|in|lb|oz|ml|L))$', answer)
    if unit_match:
        return normalize_number(unit_match.group(1))
    currency_match = re.match(r'^\\?\$?([\d,]+\.?\d*)$', answer)
    if currency_match:
        return normalize_number(currency_match.group(1))
    if re.match(r'^-?[\d,]+$', answer):
        return normalize_number(answer)
    unit_match = re.match(r'^(-?[\d,]+(?:\.\d*)?)\s*(?:\\(?:mbox|text|hbox|displaystyle)\{[^}]+\})?(?:\^?\d)?$', answer)
    if unit_match:
        return normalize_number(unit_match.group(1))
    mc_match = re.match(r'^\\text{\(?([A-Za-z])\)?}$|^\(?([A-Za-z])\)?$', answer)
    if mc_match:
        return (mc_match.group(1) or mc_match.group(2)).lower()
    degree_match = re.match(r'^(-?[\d,]+(?:\.\d*)?)\s*(?:(?:\^?\\circ)|(?:{\\circ})|(?:°))?$', answer)
    if degree_match:
        return normalize_number(degree_match.group(1))
    answer = re.sub(r'\\text{([^{}]+)}', r'\1', answer)
    try:
        return normalize_algebraic_expression(answer)
    except:
        pass
    answer = answer.replace('\\left', '').replace('\\right', '')
    answer = answer.replace('\\(', '(').replace('\\)', ')')
    answer = answer.replace('\\[', '[').replace('\\]', ']')
    answer = answer.replace('\\{', '{').replace('\\}', '}')
    answer = re.sub(r'\\sqrt\{?(\d+)\}?', r'\\sqrt{\1}', answer)
    answer = re.sub(r'\\sqrt{([^{}]+)}', r'\\sqrt\1', answer)
    if re.match(r'^\d+\\%$', answer) or re.match(r'^\d+$', answer):
        answer = re.sub(r'\\%$', '', answer)
    answer = re.sub(r'\\text{([^{}]+)}', r'\1', answer)
    while len(answer) >= 2 and answer[0] == '{' and answer[-1] == '}':
        if '\\frac' in answer:
            break
        answer = answer[1:-1]
    return answer.lower() if answer else None

def compare_answers(correct_answer: str, predicted_answer: Optional[str]) -> bool:
    if predicted_answer is None:
        return False
    if numerically_equal(correct_answer, predicted_answer):
        return True
    normalized_correct = normalize_answer(correct_answer)
    normalized_predicted = normalize_answer(predicted_answer)
    if not normalized_correct or not normalized_predicted:
        return False
    if normalized_correct == "" and normalized_predicted == "":
        return False
    if ('\\left[' in normalized_correct or '\\left(' in normalized_correct) and \
       ('\\left[' in normalized_predicted or '\\left(' in normalized_predicted):
        return normalized_correct == normalized_predicted
    return normalized_correct == normalized_predicted

# Load existing results
def load_existing_results(filename: str) -> list[Dict]:
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return []

# Save a single result
def save_result(filename: str, result: Dict):
    results = load_existing_results(filename)
    results.append(result)
    with open(filename, 'w') as f:
        json.dump(results, f, indent=2)

# Analyze and print results
def analyze_results(results: list[Dict]):
    total = len(results)
    correct = sum(1 for r in results if r['is_correct'])
    accuracy = correct / total if total > 0 else 0
    print("\n=== Results Summary ===")
    print(f"Total problems: {total}")
    print(f"Correct answers: {correct}")
    print(f"Accuracy: {accuracy:.2%}")
    print("\n=== Incorrect Problems ===")
    for r in results:
        if not r['is_correct']:
            print(f"Problem {r['index']}:")
            print(f"Expected: {r['correct_answer']}")
            print(f"Predicted: {r['predicted_answer']}")
            print("---")

# Main evaluation function
def evaluate():
    os.makedirs("results", exist_ok=True)
    results_file = "evaluation_results_math500_deepseek.json"
    dataset = load_math500_dataset()
    existing_results = load_existing_results(results_file)
    processed_indexes = {result['index'] for result in existing_results}
    cnt = 0
    t=0
    for idx, item in enumerate(tqdm(dataset, desc="Evaluating problems")):
        if idx in processed_indexes:
            continue
        t += 1
        problem_text = item['problem']
        correct_answer = extract_answer(item['solution'])  # Extract from 'solution', not 'answer'
        response = get_llm_response(problem_text)
        predicted_answer = extract_answer(response)
        is_correct = compare_answers(correct_answer, predicted_answer)
        result = {
            "index": idx,
            "problem": problem_text,
            "response": response,
            "correct_answer": correct_answer,
            "predicted_answer": predicted_answer,
            "is_correct": is_correct
        }
        save_result(results_file, result)
        if is_correct:
          cnt += 1
        print(f"cnt :  {cnt} idx: {t}")
    final_results = load_existing_results(results_file)
    analyze_results(final_results)



# Customizable CoT Prompt Template
* modify cot prompt then evaluate on math benchmark


In [ ]:

# final answer should be in this format: (because of extract_answer function you can change it if you want)
#\[
#oxed{your_answer_here}
#\]

COT_PROMPT = SYSTEM_PROMPT


* generate response with cot prompt

In [2]:
import requests

def get_COT_response(problem):
    prompt = COT_PROMPT + "\n" + problem
    url = "http://localhost:8000/v1/chat/completions"

    payload = {
        "model": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }

        ],
    "max_tokens": 1900,
    "temperature": 0.3
    }
    response = requests.post(url, json=payload)
    return response.json()['choices'][0]['message']['content'].strip()

# Evaluate CoT
* modify response generation part to evalute this method.

In [5]:

def evaluate_cot():
    os.makedirs("results", exist_ok=True)
    results_file = "evaluation_results_math500_deepseek_cot.json"
    dataset = load_math500_dataset()
    existing_results = load_existing_results(results_file)
    processed_indexes = {result['index'] for result in existing_results}
    cnt = 0
    for idx, item in enumerate(tqdm(dataset, desc="Evaluating problems")):
        if idx in processed_indexes:
            continue
        if idx >= 30:
            break
        problem_text = item['problem']
        correct_answer = extract_answer(item['solution'])
        response = get_COT_response(problem_text)
        predicted_answer = extract_answer(response)
        is_correct = compare_answers(correct_answer, predicted_answer)
        result = {
            "index": idx,
            "problem": problem_text,
            "response": response,
            "correct_answer": correct_answer,
            "predicted_answer": predicted_answer,
            "is_correct": is_correct
        }
        save_result(results_file, result)
        if is_correct:
            cnt += 1
        print(f"corrects :  {cnt} idx: {idx}")
    final_results = load_existing_results(results_file)
    analyze_results(final_results)


In [ ]:
evaluate_cot()

## Best-of-N 

The Best-of-N approach generates several candidate responses for a problem and then selects the one with the highest average token log-likelihood. This ensures that the final answer, formatted within the `\boxed{}` command, is not only correct in presentation but also statistically the most reliable.


In [13]:

SYSTEM_PROMPT = You are solving mathematics problems.

Please think step by step.

Important: Always end your solution with the final answer in this format:

\[
\boxed{your_answer_here}
\]

The entire answer should be contained completely within the \boxed{} command.


def best_of_n_response(problem, N=5):
    prompt = SYSTEM_PROMPT + "
" + problem
    responses = []
    for _ in range(N):
        resp = get_llm_response(prompt)
        ans = extract_answer(resp)
        responses.append((resp, ans, 0.0))

    # Group by answer and pick first occurrence
    best = None
    for resp, ans, score in responses:
        if ans is None:
            continue
        best = (resp, ans, score)
        break
    if best is None:
        best = responses[0] if responses else ("", None, 0.0)
    return best[1]


# Evaluate best of n

* modify response generation part to evalute this method.

In [14]:

def evaluate_best_of_n():
    os.makedirs("results", exist_ok=True)
    results_file = "evaluation_results_math500_deepseek_best_of_n.json"
    dataset = load_math500_dataset()
    existing_results = load_existing_results(results_file)
    processed_indexes = {result['index'] for result in existing_results}
    cnt = 0
    for idx, item in enumerate(tqdm(dataset, desc="Evaluating problems")):
        if idx in processed_indexes:
            continue
        if idx >= 30:
            break
        problem_text = item['problem']
        correct_answer = extract_answer(item['solution'])
        response = best_of_n_response(problem_text, N=3)
        predicted_answer = response
        is_correct = compare_answers(correct_answer, predicted_answer)
        result = {
            "index": idx,
            "problem": problem_text,
            "response": response,
            "correct_answer": correct_answer,
            "predicted_answer": predicted_answer,
            "is_correct": is_correct
        }
        save_result(results_file, result)
        if is_correct:
            cnt += 1
        print(f"corrects :  {cnt} idx: {idx}")
    final_results = load_existing_results(results_file)
    analyze_results(final_results)


In [ ]:
evaluate_best_of_n()

## Beam Search

This cell implements a beam search strategy for generating candidate reasoning chains. The method generates multiple continuations at each reasoning step, scoring each candidate based on its average token log-likelihood. By retaining and expanding only the top candidates, the approach efficiently searches for the most promising chain-of-thought that leads to the final answer in the required format.


In [25]:

def call_qwen_model_raw(prompt, step_num, temperature=0.8):
    output_text = get_llm_response(prompt)
    token_logprobs = [0.0 for _ in output_text.split()] or [0.0]
    avg_token_prob = sum(token_logprobs) / len(token_logprobs)
    return output_text, avg_token_prob, len(token_logprobs)


class BeamCandidate:
    def __init__(self, sequence, cumulative_log_prob, step_scores, finished=False, num_token=0):
        self.sequence = sequence
        self.cumulative_log_prob = cumulative_log_prob
        self.step_scores = step_scores
        self.finished = finished
        self.num_token = num_token

    def __repr__(self):
        return (f"BeamCandidate(score={self.cumulative_log_prob:.3f}, finished={self.finished}, "
                f"sequence={self.sequence})")


def generate_reasoning_steps(context, step_num, top_k):
    candidates = []
    for _ in range(top_k):
        candidate_prompt = context + f"
Step {step_num}:"
        output_text, avg_token_prob, num_token = call_qwen_model_raw(candidate_prompt, step_num)
        finished = "\boxed{" in output_text
        candidate_step = candidate_prompt + "
" + output_text
        candidates.append((candidate_step, avg_token_prob, num_token, finished))
    return candidates


def beam_search(init_problem_prompt, beam_width=3, max_steps=3, top_k=2):
    initial_candidate = BeamCandidate(init_problem_prompt, 0.0, [], finished=False)
    beams = [initial_candidate]

    for step_num in range(1, max_steps + 1):
        new_beams = []
        for candidate in beams:
            if candidate.finished:
                new_beams.append(candidate)
                continue
            step_candidates = generate_reasoning_steps(candidate.sequence, step_num, top_k)
            for (step_text, score, num_token, finished) in step_candidates:
                new_score = (candidate.cumulative_log_prob * len(candidate.step_scores) + score) / (len(candidate.step_scores) + 1)
                new_beams.append(BeamCandidate(step_text, new_score, candidate.step_scores + [score], finished, num_token))
        if not new_beams:
            break
        new_beams.sort(key=lambda c: c.cumulative_log_prob, reverse=True)
        beams = new_beams[:beam_width]
        if all(beam.finished for beam in beams):
            break
    finished_beams = [b for b in beams if b.finished] or beams
    best_candidate = max(finished_beams, key=lambda b: b.cumulative_log_prob)
    return best_candidate


def run_qwen_beam_search(problem, beam_width, max_steps, top_k, log_level):
    prompt = SYSTEM_PROMPT + "
" + problem
    best_candidate = beam_search(prompt, beam_width=beam_width, max_steps=max_steps, top_k=top_k)
    final_answer = extract_answer(best_candidate.sequence)
    return final_answer


# Evaluate beam search
* modify response generation part to evalute this method.

In [26]:

def evaluate_beam_search():
    os.makedirs("results", exist_ok=True)
    results_file = "evaluation_results_math500_deepseek_beam_search.json"
    dataset = load_math500_dataset()
    existing_results = load_existing_results(results_file)
    processed_indexes = {result['index'] for result in existing_results}
    cnt = 0
    for idx, item in enumerate(tqdm(dataset, desc="Evaluating problems")):
        if idx in processed_indexes:
            continue
        if idx >= 30:
            break
        problem_text = item['problem']
        correct_answer = extract_answer(item['solution'])
        response = run_qwen_beam_search(problem_text, beam_width=2, max_steps=2, top_k=2, log_level=0)
        predicted_answer = response
        is_correct = compare_answers(correct_answer, predicted_answer)
        result = {
            "index": idx,
            "problem": problem_text,
            "response": response,
            "correct_answer": correct_answer,
            "predicted_answer": predicted_answer,
            "is_correct": is_correct
        }
        save_result(results_file, result)
        if is_correct:
            cnt += 1
        print(f"corrects :  {cnt} idx: {idx}")
    final_results = load_existing_results(results_file)
    analyze_results(final_results)


In [ ]:
evaluate_beam_search()

## Self-Refinement 

This approach begins by generating an initial solution using the given prompt. It then iteratively refines this output by providing the model with targeted feedback and asking it to improve its response. The process continues until the feedback indicates that no further refinement is necessary, ensuring that the final answer—properly formatted within the `\boxed{}` command—is as accurate and well-reasoned as possible.


In [19]:

SYSTEM_PROMPT = You are solving mathematics problems.

Please think step by step.

Important: Always end your solution with the final answer in this format:

\[
\boxed{your_answer_here}
\]

The entire answer should be contained completely within the \boxed{} command.


def generate_content(prompt):
    output_text = get_llm_response(prompt)
    return output_text


def self_refine(problem, max_iter=2):
    prompt = SYSTEM_PROMPT + "
" + problem
    current_output = generate_content(prompt)

    for iteration in range(max_iter):
        feedback_prompt = (
            SYSTEM_PROMPT
            + "
Here is a candidate solution. If it is correct, say 'OK'.
"
            + problem
            + "
Candidate:
"
            + current_output
        )
        feedback = generate_content(feedback_prompt)
        if 'OK' in feedback:
            break
        refine_prompt = (
            SYSTEM_PROMPT
            + "
Improve the solution based on feedback.
Problem:
"
            + problem
            + "
Current solution:
"
            + current_output
            + "
Feedback:
"
            + feedback
        )
        current_output = generate_content(refine_prompt)

    answer = extract_answer(current_output)
    return answer


# Evaluate Self-Refinement
* modify response generation part to evalute this method.

In [20]:

def evaluate_self_refiner():
    os.makedirs("results", exist_ok=True)
    results_file = "evaluation_results_math500_deepseek_self_refiner.json"
    dataset = load_math500_dataset()
    existing_results = load_existing_results(results_file)
    processed_indexes = {result['index'] for result in existing_results}
    cnt = 0
    for idx, item in enumerate(tqdm(dataset, desc="Evaluating problems")):
        if idx in processed_indexes:
            continue
        if idx >= 30:
            break
        problem_text = item['problem']
        correct_answer = extract_answer(item['solution'])
        response = self_refine(problem_text, max_iter=2)
        predicted_answer = response
        is_correct = compare_answers(correct_answer, predicted_answer)
        result = {
            "index": idx,
            "problem": problem_text,
            "response": response,
            "correct_answer": correct_answer,
            "predicted_answer": predicted_answer,
            "is_correct": is_correct
        }
        save_result(results_file, result)
        if is_correct:
            cnt += 1
        print(f"corrects :  {cnt} idx: {idx}")
    final_results = load_existing_results(results_file)
    analyze_results(final_results)


In [ ]:
evaluate_self_refiner()